## Hypothesis Testing

In [13]:
import pandas as pd
import numpy as np
from scipy import stats
import itertools

## Null and Alternative Hypotheses


A toy picture of science has it that the scientist formulates a hypothesis that explains or generalizes from some set of observations, and then conducts some experiment, which will either confirm or refute that hypothesis.



But this is an oversimplification of good practice. Consider the possibility that the experiment yield statistically improbable results. In that case it may well be a mistake to generalize from those results or to reject an hypothesis that doesn't predict them.


Often the confirmation of some testing or alternative hypothesis, $H_\alpha$, is a relative affair, where it is measured against some null hypothesis, $H_0$.


If an alternative hypothesis states that there is some significant relationship between two variables, then the null hypothesis simply states that there is no such relationship.


If we're testing the function of a new drug, then the null hypothesis will say that the drug has no effect on patients, or anyway no effect relative to relief of the malady the drug was designed to combat. If we're testing whether Peeps cause dementia, then the null hypothesis will say that there is no correlation between Peeps consumption and rate of dementia development.

## $p$-Values


The basic idea of a p-value is to quantify the probability that the results seen are in fact the result of mere random chance. This is connected with the null hypothesis since, if the null hypothesis is true and there is no significant correlation between the population variables X and Y, then of course any correlation between X and Y observed in our sample would have to be the result of mere random chance.

## How Unlikely Is Too Unlikely?


Suppose we calculate a p-value for some statistic we've measured (more on this below!) and we get a p-value of 20%. This would mean that there is a 20% chance that the results we observed were the result of mere random chance. Probably this is high enough that we ought not to reject the null hypothesis that our variables are uncorrelated.

In practice, a p-value threshold of 5% is very often the default value for these tests of statistical significance. Thus, if it is calculated that the chance that the results we observed were actually the result of randomness is less than 1 in 20, then we would reject the null hypothesis and accept the alternative hypothesis.

## An Example


Let's work with the normal distribution, since it's so useful. Suppose we are told that African elephants have weights distributed normally around a mean of 9000 lbs., with a standard deviation of 900 lbs. Pachyderm Adventures has recently measured the weights of 25 African elephants in Gabon and has calculated their average weight at 8637 lbs. They claim that these statistics on the Gabonese elephants are significant. Let's find out!

What is our null hypothesis here?

In [14]:
# NH: The null hypothesis is that the Gabonese elephants' weights are
# no different from the general population.

Let's set a threshold value of $p = 0.05$ for rejecting the null hypothesis.

The standard error for our sample is: $\large\frac{\sigma}{\sqrt{n}} = \frac{900}{\sqrt{25}} = 180$.

In [25]:
se = 900 / np.sqrt(25)
se

180.0


The z-score will be: $\large\frac{\bar{x} - \mu}{se} = \frac{8637 - 9000}{180} = -\frac{363}{180} = -2.02$.

In [16]:
z = (8637 - 9000) / 180
z

-2.0166666666666666

In [17]:
stats.norm.sf(z) # This is the "survival function": 1 - cdf(z)


0.97813484463207

The "area of the tail" corresponding to this z-score is only 2.2%. The area for both tails is thus 4.4%, still less than our 5% threshold value.

Thus we ought to reject the null hypothesis and conclude that the Gabonese elephants are statistically significant!

## Type 1 Errors (False Positives) and Type 2 Errors (False Negatives)

<img src="confusion_matrix_simple2.png">

Most tests for the presence of some factor are imperfect. And in fact most tests are imperfect in two ways: They will sometimes fail to predict the presence of that factor when it is after all present, and they will sometimes predict the presence of that factor when in fact it is not. Clearly, the lower these error rates are, the better, but it is not uncommon for these rates to be between 1% and 5%, and sometimes they are even higher than that. (Of course, if they're higher than 50%, then we're better off just flipping a coin to run our test!)

Predicting the presence of some factor (i.e. counter to the null hypothesis) when in fact it is not there (i.e. the null hypothesis is true) is called a "false positive". Failing to predict the presence of some factor (i.e. in accord with the null hypothesis) when in fact it is there (i.e. the null hypothesis is false) is called a "false negative".



## What is a Confidence Interval?
A Confidence Interval is a range of values above and below the point estimate that captures the true population parameter at some predetermined confidence level. If we want to have a 95% chance of capturing the true population parameter with a point estimate and a corresponding confidence interval, we would set confidence level to 95%. Higher confidence levels result in a wider confidence intervals.

We calculate a confidence interval by taking a point estimate and then adding and subtracting a margin of error to create a range. Margin of error is based on your desired confidence level, the spread of the data and the size of your sample. The way you calculate the margin of error depends on whether you know the standard deviation of the population or not.

the margin of error for a known population stadard deviation is:

** Margin of Error = z ∗ σ / √n**

Where σ (sigma) is the population standard deviation, n is sample size, and z is a number known as the z-critical value.

The z-critical value is the number of standard deviations you'd have to go from the mean of the normal distribution to capture the proportion of the data associated with the desired confidence level.

For instance, we know that roughly 95% of the data in a normal distribution lies within 2 standard deviations of the mean, so we could use 2 as the z-critical value for a 95% confidence interval as shown in this image: 

<img src="confdiag.gif">


In [23]:
import numpy as np
import scipy.stats


def mean_confidence_interval(data, confidence=0.95):
    a = np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2, n-1)
    return m, m-h, m+h


In [24]:
data = [3,4,6,2,1,5,7,9]
mean_confidence_interval(data)

(4.625, 2.3934348004057675, 6.8565651995942325)

## Statistical significance versus practical significance
When the sample size becomes larger, point estimates become more precise and any real differ- ences in the mean and null value become easier to detect and recognize. Even a very small difference would likely be detected if we took a large enough sample. Sometimes researchers will take such large samples that even the slightest difference is detected, even differences where there is no practical value. In such cases, we still say the difference is statistically significant, but it is not practi- cally significant. For example, an online experiment might identify that placing additional ads on a movie review website statistically significantly increases viewership of a TV show by 0.001%, but this increase might not have any practical value.
One role of a data scientist in conducting a study often includes planning the size of the study. The data scientist might first consult experts or scientific literature to learn what would be the smallest meaningful difference from the null value. She also would obtain other information, such as a very rough estimate of the true proportion p, so that she could roughly estimate the standard error. From here, she can suggest a sample size that is sufficiently large that, if there is a real difference that is meaningful, we could detect it. While larger sample sizes may still be used, these calculations are especially helpful when considering costs or potential risks, such as possible health impacts to volunteers in a medical study.


## One-sided hypothesis tests (special topic)
So far we’ve only considered what are called two-sided hypothesis tests, where we care about detecting whether p is either above or below some null value p0. There is a second type of hypothesis test called a one-sided hypothesis test. For a one-sided hypothesis test, the hypotheses take one of the following forms:
1. There’s only value in detecting if the population parameter is less than some value p0. In this case, the alternative hypothesis is written as p < p0 for some null value p0.
2. There’s only value in detecting if the population parameter is more than some value p0: In this case, the alternative hypothesis is written as p > p0.
While we adjust the form of the alternative hypothesis, we continue to write the null hypothesis using an equals-sign in the one-sided hypothesis test case.
In the entire hypothesis testing procedure, there is only one difference in evaluating a one- sided hypothesis test vs a two-sided hypothesis test: how to compute the p-value. In a one-sided hypothesis test, we compute the p-value as the tail area in the direction of the alternative hypothesis only, meaning it is represented by a single tail area. Herein lies the reason why one-sided tests are sometimes interesting: if we don’t have to double the tail area to get the p-value, then the p-value is smaller and the level of evidence required to identify an interesting finding in the direction of the alternative hypothesis goes down. However, one-sided tests aren’t all sunshine and rainbows: the heavy price paid is that any interesting findings in the opposite direction must be disregarded.

## Let's look at a problem that is evaluating the efficacy of a medical treatment. 
In this section we will consider an experiment that studies effectiveness of stents in treating patients at risk of stroke. Stents are devices put inside blood vessels that assist in patient recovery after cardiac events and reduce the risk of an additional heart attack or death. Many doctors have hoped that there would be similar benefits for patients at risk of stroke. We start by writing the principal question the researchers hope to answer:
Does the use of stents reduce the risk of stroke?
The researchers who asked this question conducted an experiment with 451 at-risk patients. Each volunteer patient was randomly assigned to one of two groups:
Treatment group. Patients in the treatment group received a stent and medical manage- ment. The medical management included medications, management of risk factors, and help in lifestyle modification.
Control group. Patients in the control group received the same medical management as the treatment group, but they did not receive stents.
Researchers randomly assigned 224 patients to the treatment group and 227 to the control group. In this study, the control group provides a reference point against which we can measure the medical impact of stents in the treatment group.
Researchers studied the effect of stents at two time points: 30 days after enrollment and 365 days after enrollment. The results of 5 patients are summarized in Figure 1.1. Patient outcomes are recorded as “stroke” or “no event”, representing whether or not the patient had a stroke at the end of a time period.

0-365 days with stents: 
    Stroke 45 
    No Stroke 179
    SSD: .4
0-365 days w/o stents:
    Stroke 28
    No Stroke 199
    SSD: .8


2 SIDED at 95 percent confidence: 
CV: +-1.962

(X2-X1)-(mu1-mu2)/sqrt(ssd1/n1=ssd2/n2)

In [8]:
import numpy as np
X1=45
n1 = 45+179
mu_1=45/n1
ssd_1 = .4

X2 = 28
n2 = 28+199
mu_2=28/n2
ssd_2 = .8

In [7]:
(X2-X1)-(mu_2-mu_1)/np.sqrt(ssd_1/n1+ssd_2/n2)

-15.935837484156517


We encountered an example where doctors were interested in determining whether stents would help people who had a high risk of stroke. The researchers believed the stents would help. Unfortunately, the data showed the opposite: patients who received stents actually did worse. Why was using a two-sided test so important in this context?
Before the study, researchers had reason to believe that stents would help patients since existing research suggested stents helped in patients with heart attacks. It would surely have been tempting to use a one-sided test in this situation, and had they done this, they would have limited their ability to identify potential harm to patients.
 Example 5.37 highlights that using a one-sided hypothesis creates a risk of overlooking data supporting the opposite conclusion. We could have made a similar error when reviewing the Roslings’ question data this section; if we had a pre-conceived notion that college-educated people wouldn’t do worse than random guessing and so used a one-sided test, we would have missed the really interesting finding that many people have incorrect knowledge about global public health.
When might a one-sided test be appropriate to use? Very rarely. Should you ever find yourself considering using a one-sided test, carefully answer the following question:
What would I, or others, conclude if the data happens to go clearly in the opposite direc- tion than my alternative hypothesis?
If you or others would find any value in making a conclusion about the data that goes in the opposite direction of a one-sided test, then a two-sided hypothesis test should actually be used. These considerations can be subtle, so exercise caution. We will only apply two-sided tests in the rest of this book.